# User-defined functions

Si bien Apache Spark cuenta con una gran cantidad de funciones incorporadas, la flexibilidad de Spark permite que los ingenieros de datos y los científicos de datos también definan sus propias funciones. Estas son conocidas como funciones definidas por el usuario (UDFs).


El beneficio de crear tus propias UDFs (funciones definidas por el usuario) en PySpark o Scala la reusabilidad de dichas funciones para todos los componentes del equipo de desarrollo. Por ejemplo, un científico de datos puede introducir un modelo de aprendizaje automático (ML) dentro de una UDF para que un analista de datos pueda consultar sus predicciones en Spark sin necesidad de entender los detalles internos del modelo.

In [ ]:
%pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## Primer ejemplo básico -> Elevar al cubo

In [ ]:
from pyspark.sql.types import LongType

# Función para elevar al cubo
def cube(s):
  return s * s * s

# Registro de la UDF
spark.udf.register('cube', cube, LongType())

<function __main__.cube(s)>

In [ ]:
spark.range(1, 10).createOrReplaceTempView('udf_test')

In [ ]:
spark.sql('select * from udf_test').show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [ ]:
df = spark.sql('select id, cube(id) as cube from udf_test')

In [ ]:
df.show()

+---+----+
| id|cube|
+---+----+
|  1|   1|
|  2|   8|
|  3|  27|
|  4|  64|
|  5| 125|
|  6| 216|
|  7| 343|
|  8| 512|
|  9| 729|
+---+----+



## Aceleración de UDFs con UDFs de Pandas

Uno de los problemas al utilizar UDFs de PySpark era que tenían un rendimiento más lento que las UDFs de Scala. Esto se debía a que las UDFs de PySpark requerían el movimiento de datos entre el JVM y Python, lo cual era bastante costoso. Para resolver este problema, se introdujeron las UDFs de Pandas (también conocidas como UDFs vectorizadas) como parte de Apache Spark 2.3. Una UDF de Pandas utiliza Apache Arrow para transferir datos y Pandas para trabajar con los datos.

Una UDF de Pandas se define utilizando la palabra clave pandas_udf como decorador. En lugar de operar en entradas individuales fila por fila, se opera en una Serie de Pandas (realiza una ejecución vectorizada).

In [ ]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

@pandas_udf('integer')
def add_one(s: pd.Series) -> pd.Series:
  return s + 1

spark.udf.register("add_one", add_one)
spark.sql("SELECT add_one(id) FROM udf_test").show()

+-----------+
|add_one(id)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
|         10|
+-----------+



## Otro ejemplo -> con varias columnas

In [ ]:
from pyspark.sql.functions import pandas_udf, round
from pyspark.sql.types import DoubleType

@pandas_udf(DoubleType())
def calculate_ratio(column1, column2):
    return column1 / column2

df.withColumn('ratio_column', round(calculate_ratio(df['id'], df['cube']), 2)).show()

+---+----+------------+
| id|cube|ratio_column|
+---+----+------------+
|  1|   1|         1.0|
|  2|   8|        0.25|
|  3|  27|        0.11|
|  4|  64|        0.06|
|  5| 125|        0.04|
|  6| 216|        0.03|
|  7| 343|        0.02|
|  8| 512|        0.02|
|  9| 729|        0.01|
+---+----+------------+



## Otro ejemplo con una función personalizada

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType

def my_func(x):
  if x % 2 == 0:
    return 0
  return x * x

@pandas_udf(StructType([StructField('result', IntegerType())]))
def process_text(series):
    processed_data = series.apply(lambda x: my_func(x))
    return pd.DataFrame({'result': processed_data})

df.withColumn('processed_column', process_text(df['id'])).show()

+---+----+----------------+
| id|cube|processed_column|
+---+----+----------------+
|  1|   1|             {1}|
|  2|   8|             {0}|
|  3|  27|             {9}|
|  4|  64|             {0}|
|  5| 125|            {25}|
|  6| 216|             {0}|
|  7| 343|            {49}|
|  8| 512|             {0}|
|  9| 729|            {81}|
+---+----+----------------+



Para ampliar conocimientos acerca de UDFs podéis acceder al siguiente enlace de documentación de Databricks:

https://docs.databricks.com/en/udf/pandas.html